In [13]:
# Importing necessary libraries
import numpy as np
import librosa
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, SimpleRNN, BatchNormalization
from keras.optimizers import Adam

In [24]:
def extract_mfccs(audio_files):
    mfccs_features = []
    
    for file_path in audio_files:
        try:
            audio, _ = librosa.load(file_path, res_type='kaiser_fast', duration=30) 
            mfccs = librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=13)
            mfccs_processed = np.mean(mfccs.T, axis=0)
            mfccs_features.append(mfccs_processed)
            
        except Exception as e:
            print("Error encountered while processing file:", file_path)
            print(e)
            continue
    
    return mfccs_features

def extract_spectrogram(audio_files):
    spectrogram_features = []
    
    for file_path in audio_files:
        try:
            audio, _ = librosa.load(file_path, res_type='kaiser_fast', duration=30) 
            spectrogram = np.abs(librosa.stft(audio))
            spectrogram_features.append(spectrogram)
            
        except Exception as e:
            print("Error encountered while processing file:", file_path)
            continue
    
    return np.array(spectrogram_features)

def extract_chroma(audio_files):
    chroma_features = []
    
    for file_path in audio_files:
        try:
            audio, _ = librosa.load(file_path, res_type='kaiser_fast', duration=30) 
            chroma = librosa.feature.chroma_stft(y=audio, sr=22050)
            chroma_features.append(chroma)
            
        except Exception as e:
            print("Error encountered while processing file:", file_path)
            print(e)
            continue
    
    return np.array(chroma_features)


In [25]:
def load_data(data_path, feature_type):
    features = []
    labels = []
    
    try:
        for folder in os.listdir(data_path):
            genre_folder = os.path.join(data_path, folder)
            if os.path.isdir(genre_folder):  # Check if it's a directory
                print("Processing folder:", genre_folder)
                for file in os.listdir(genre_folder):
                    file_path = os.path.join(genre_folder, file)
                    if os.path.isfile(file_path):  # Check if it's a file
                        print("Processing file:", file_path)
                        if feature_type == 'mfcc':
                            feature = extract_mfccs([file_path])
                            if feature:
                                features.append(feature[0])
                                labels.append(folder)
                        elif feature_type == 'spectrogram':
                            feature = extract_spectrogram([file_path])
                            if feature:
                                features.append(feature[0])
                                labels.append(folder)
                        elif feature_type == 'chroma':
                            feature = extract_chroma([file_path])
                            if feature:
                                features.append(feature[0])
                                labels.append(folder)
                    else:
                        print("Skipping non-file:", file_path)
            else:
                print("Skipping non-directory:", genre_folder)
                
    except Exception as e:
        print("Error occurred during data loading:", e)
    
    return np.array(features), np.array(labels)


In [26]:
# Load data
data_path = 'Data/genres_original'  # Update with your dataset path
#mfccs_features, mfccs_labels = load_data(data_path, feature_type='mfcc')
spectrogram_features, spectrogram_labels = load_data(data_path, feature_type='spectrogram')
#chroma_features, chroma_labels = load_data(data_path, feature_type='chroma')

Processing folder: Data/genres_original\blues
Processing file: Data/genres_original\blues\blues.00000.wav
Error occurred during data loading: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()


In [7]:
mfccs_features.shape

AttributeError: 'list' object has no attribute 'shape'

In [6]:
# Encode labels for each type of feature
label_encoder = LabelEncoder()
encoded_mfccs_labels = label_encoder.fit_transform(mfccs_labels)
encoded_spectrogram_labels = label_encoder.fit_transform(spectrogram_labels)
encoded_chroma_labels = label_encoder.fit_transform(chroma_labels)

# Convert encoded labels into categorical labels
mfccs_labels_categorical = to_categorical(encoded_mfccs_labels)
spectrogram_labels_categorical = to_categorical(encoded_spectrogram_labels)
chroma_labels_categorical = to_categorical(encoded_chroma_labels)

# Convert lists of features into NumPy arrays
mfccs_features_array = np.array(mfccs_features)
spectrogram_features_array = np.array(spectrogram_features)
chroma_features_array = np.array(chroma_features)

# Print shapes of features and labels with feature types indicated
print("MFCCs features shape:", mfccs_features_array.shape, "(MFCCs)")
print("Spectrogram features shape:", spectrogram_features_array.shape, "(Spectrogram)")
print("Chroma features shape:", chroma_features_array.shape, "(Chroma)")
print("MFCCs Labels shape:", mfccs_labels_categorical.shape)
print("Spectrogram Labels shape:", spectrogram_labels_categorical.shape)
print("Chroma Labels shape:", chroma_labels_categorical.shape)

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
# Split data into train and test sets
X_train_mfccs, X_test_mfccs, y_train_mfccs, y_test_mfccs = train_test_split(mfccs_features_array, mfccs_labels_categorical, test_size=0.2, random_state=42)
X_train_spectrogram, X_test_spectrogram, y_train_spectrogram, y_test_spectrogram = train_test_split(spectrogram_features_array, spectrogram_labels_categorical, test_size=0.2, random_state=42)
X_train_chroma, X_test_chroma, y_train_chroma, y_test_chroma = train_test_split(chroma_features_array, chroma_labels_categorical, test_size=0.2, random_state=42)

# Print shapes of the split sets
print("MFCCs features - Train shape:", X_train_mfccs.shape, "Test shape:", X_test_mfccs.shape)
print("Spectrogram features - Train shape:", X_train_spectrogram.shape, "Test shape:", X_test_spectrogram.shape)
print("Chroma features - Train shape:", X_train_chroma.shape, "Test shape:", X_test_chroma.shape)
print("MFCCs Labels shape - Train:", y_train_mfccs.shape, "Test:", y_test_mfccs.shape)
print("Spectrogram Labels shape - Train:", y_train_spectrogram.shape, "Test:", y_test_spectrogram.shape)
print("Chroma Labels shape - Train:", y_train_chroma.shape, "Test:", y_test_chroma.shape)

In [ ]:
# Reshape input data to have the appropriate shape for RNN for each set of features
X_train_mfccs_reshaped = X_train_mfccs.reshape(X_train_mfccs.shape[0], X_train_mfccs.shape[1], 1)
X_test_mfccs_reshaped = X_test_mfccs.reshape(X_test_mfccs.shape[0], X_test_mfccs.shape[1], 1)

X_train_spectrogram_reshaped = X_train_spectrogram.reshape(X_train_spectrogram.shape[0], X_train_spectrogram.shape[1], 1)
X_test_spectrogram_reshaped = X_test_spectrogram.reshape(X_test_spectrogram.shape[0], X_test_spectrogram.shape[1], 1)

X_train_chroma_reshaped = X_train_chroma.reshape(X_train_chroma.shape[0], X_train_chroma.shape[1], 1)
X_test_chroma_reshaped = X_test_chroma.reshape(X_test_chroma.shape[0], X_test_chroma.shape[1], 1)

# Print shapes of the reshaped sets
print("MFCCs features - Train shape (reshaped):", X_train_mfccs_reshaped.shape, "Test shape (reshaped):", X_test_mfccs_reshaped.shape)
print("Spectrogram features - Train shape (reshaped):", X_train_spectrogram_reshaped.shape, "Test shape (reshaped):", X_test_spectrogram_reshaped.shape)
print("Chroma features - Train shape (reshaped):", X_train_chroma_reshaped.shape, "Test shape (reshaped):", X_test_chroma_reshaped.shape)

In [ ]:
# Define a function to create the model
def create_model(input_shape, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(SimpleRNN(units=128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(BatchNormalization())
    model.add(SimpleRNN(units=64, return_sequences=True))
    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(BatchNormalization())
    model.add(SimpleRNN(units=32))
    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(BatchNormalization())
    model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize variables to store best score and parameters for each feature set
best_scores = {'mfccs': 0, 'spectrogram': 0, 'chroma': 0}
best_params = {'mfccs': {}, 'spectrogram': {}, 'chroma': {}}

# Define lists of hyperparameters to search over
learning_rates = [0.001, 0.0015, 0.002]
dropout_rates = [0.1, 0.2]
batch_sizes = [16, 32, 64, 128]
epochs_list = [30, 40, 50, 60]

# Iterate over all combinations of hyperparameters for each feature set
for feature_type, X_train, X_test in [('mfccs', X_train_mfccs_reshaped, X_test_mfccs_reshaped),
                                      ('spectrogram', X_train_spectrogram_reshaped, X_test_spectrogram_reshaped),
                                      ('chroma', X_train_chroma_reshaped, X_test_chroma_reshaped)]:

    for lr in learning_rates:
        for dropout_rate in dropout_rates:
            for batch_size in batch_sizes:
                for epochs in epochs_list:
                    # Create model with current hyperparameters
                    model = create_model(input_shape=X_train.shape[1:], learning_rate=lr, dropout_rate=dropout_rate)
                    
                    # Train the model
                    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
                    
                    # Evaluate the model on validation data
                    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
                    
                    # Store the accuracy and parameters if it's the best for the current feature set
                    if accuracy > best_scores[feature_type]:
                        best_scores[feature_type] = accuracy
                        best_params[feature_type] = {'learning_rate': lr, 'dropout_rate': dropout_rate, 'batch_size': batch_size, 'epochs': epochs}
                    
                    # Print accuracy and parameters for each iteration
                    params = {'learning_rate': lr, 'dropout_rate': dropout_rate, 'batch_size': batch_size, 'epochs': epochs}
                    print(f"Feature Type: {feature_type}, Accuracy: {accuracy}, Parameters: {params}")

# Print the best scores and parameters for each feature set
for feature_type in best_scores:
    print(f"Best Mean Test Accuracy ({feature_type}):", best_scores[feature_type])
    print(f"Best Parameters ({feature_type}):", best_params[feature_type])
